In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("amazon_dataset.csv")

In [3]:
df.shape

(94593, 12)

In [4]:
df.head(5)

,Unnamed: 0,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,eco_friendly,description
0,8994,MyBlush Women Top,women's clothing,Western Wear,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/MyBlush-Womens-Solid-Cla...,3.5,9,₹699,"₹2,299",No,MyBlush Women Top women's clothing Western Wear
1,6994,Ridhi Home & Decor Brass Decorative Weighing S...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Brass-Decorative-Weighin...,3.7,64,₹380,₹500,Yes,Ridhi Home & Decor Brass Decorative Weighing S...
2,7494,Rangshaily Men's Black White Floral Print Full...,men's clothing,Shirts,https://m.media-amazon.com/images/I/51oiWRu+Gp...,https://www.amazon.in/Rangshaily-Regular-Flora...,NaN,NaN,₹649,₹999,Yes,Rangshaily Men's Black White Floral Print Full...
3,5066,Adidas Men's Ellipto Run M Leather Shoe,men's shoes,Sports Shoes,https://m.media-amazon.com/images/I/71RsZ19e-w...,https://www.amazon.in/Adidas-Ellipto-Running-D...,3.6,23,"₹1,274","₹2,499",No,Adidas Men's Ellipto Run M Leather Shoe men's ...
4,14687,JJ TEES Polyester Half Sleeve Jersey with Roun...,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/JJ-TEES-Polyester-Sleeve...,3.5,10,₹345,₹999,No,JJ TEES Polyester Half Sleeve Jersey with Roun...


In [5]:
df.isnull().sum()

,0
Unnamed: 0,0
name,0
main_category,1
sub_category,1
image,1
link,1
ratings,30228
no_of_ratings,30228
discount_price,10610
actual_price,3120


In [6]:
df.drop(["ratings", "no_of_ratings", "discount_price", "actual_price"], axis=1, inplace=True)

In [7]:
df.isnull().sum()

,0
Unnamed: 0,0
name,0
main_category,1
sub_category,1
image,1
link,1
eco_friendly,1
description,1


In [8]:
df.dropna(inplace=True)

In [9]:
df["description"] = df["description"].fillna("")

In [10]:
df.shape

(94592, 8)

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.sample()

,Unnamed: 0,name,main_category,sub_category,image,link,eco_friendly,description
17763,2464,Yellow Chimes Rings for Men Dragon Celtic Inla...,stores,Men's Fashion,https://m.media-amazon.com/images/I/61dpeFxpjK...,https://www.amazon.in/Yellow-Chimes-Dragon-Cel...,No,Yellow Chimes Rings for Men Dragon Celtic Inla...


In [13]:
df.head(5)

,Unnamed: 0,name,main_category,sub_category,image,link,eco_friendly,description
0,8994,MyBlush Women Top,women's clothing,Western Wear,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/MyBlush-Womens-Solid-Cla...,No,MyBlush Women Top women's clothing Western Wear
1,6994,Ridhi Home & Decor Brass Decorative Weighing S...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Brass-Decorative-Weighin...,Yes,Ridhi Home & Decor Brass Decorative Weighing S...
2,7494,Rangshaily Men's Black White Floral Print Full...,men's clothing,Shirts,https://m.media-amazon.com/images/I/51oiWRu+Gp...,https://www.amazon.in/Rangshaily-Regular-Flora...,Yes,Rangshaily Men's Black White Floral Print Full...
3,5066,Adidas Men's Ellipto Run M Leather Shoe,men's shoes,Sports Shoes,https://m.media-amazon.com/images/I/71RsZ19e-w...,https://www.amazon.in/Adidas-Ellipto-Running-D...,No,Adidas Men's Ellipto Run M Leather Shoe men's ...
4,14687,JJ TEES Polyester Half Sleeve Jersey with Roun...,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/JJ-TEES-Polyester-Sleeve...,No,JJ TEES Polyester Half Sleeve Jersey with Roun...


In [14]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['description'])

In [15]:
similarity = cosine_similarity(tfidf_matrix)

In [16]:
name_to_index = pd.Series(df.index, index=df['name'].str.lower()).drop_duplicates()

# Recommend All Products

In [17]:
def recommend_all_products(product_name, top_n=5):
    product_name = product_name.lower()
    if product_name not in name_to_index:
        return f" Product '{product_name}' not found."

    idx = name_to_index[product_name]
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    result = df.iloc[[i[0] for i in sim_scores]][['name', 'description', 'eco_friendly', 'link']]
    return result

# Recommend Eco-Friendly Products

In [18]:
def recommend_eco_friendly(product_name, top_n=10):
    results = recommend_all_products(product_name, top_n=top_n * 2)  # fetch more, filter later
    if isinstance(results, str):
        return results
    eco_results = results[results['eco_friendly'].str.lower() == 'yes'].head(5)
    return eco_results

In [19]:
recommend_all_products("MyBlush Women Top")


,name,description,eco_friendly,link
2349,Elleven Women Culotte,Elleven Women Culotte women's clothing Western...,No,https://www.amazon.in/Elleven-Womens-Regular-C...
9873,KOTTY Women's Top,KOTTY Women's Top women's clothing Western Wear,Yes,https://www.amazon.in/KOTTY-Womens-Printed-Top...
9996,SIDDHANAM Women Top,SIDDHANAM Women Top women's clothing Western Wear,No,https://www.amazon.in/SIDDHANAM-Trendy-Cotton-...
13038,MIDAAS Women's Top,MIDAAS Women's Top women's clothing Western Wear,No,https://www.amazon.in/MIDAAS-Womens-Cotton-Whi...
14328,Stylistico Women's Top,Stylistico Women's Top women's clothing Wester...,No,https://www.amazon.in/Stylistico-Womens-Embroi...


In [20]:
recommend_eco_friendly("MyBlush Women Top")

,name,description,eco_friendly,link
9873,KOTTY Women's Top,KOTTY Women's Top women's clothing Western Wear,Yes,https://www.amazon.in/KOTTY-Womens-Printed-Top...
22751,SAAKAA Women's Top,SAAKAA Women's Top women's clothing Western Wear,Yes,https://www.amazon.in/SAAKAA-Womens-Crepe-Pink...
37647,AELOMART Women's Top(AWT6044-Pn_Multicolour),AELOMART Women's Top(AWT6044-Pn_Multicolour) w...,Yes,https://www.amazon.in/AELOMART-Womens-AWT6044-...
39003,Elleven Women Culotte,Elleven Women Culotte women's clothing Western...,Yes,https://www.amazon.in/Elleven-21CRE60009-70133...
72423,Jwalin Women Top,Jwalin Women Top women's clothing Western Wear,Yes,https://www.amazon.in/Jwalin-Womens-Cotton-Pri...
